In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import SteamVariables as sv

scaler = StandardScaler()
df = pd.read_csv(sv.CSV_PATH, nrows=4000000)
df = df.dropna()

#print(df.groupby('app_name').first().reset_index())


feature_columnsNew= [sv.AUTHOR_STEAMID,sv.APP_NAME,sv.WEIGHTED_VOTE_SCORE]

new_dt = df[feature_columnsNew]
print(new_dt)

            author.steamid                  app_name  weighted_vote_score
0        76561199095369542  The Witcher 3: Wild Hunt             0.000000
1        76561198949504115  The Witcher 3: Wild Hunt             0.000000
2        76561199090098988  The Witcher 3: Wild Hunt             0.000000
3        76561199054755373  The Witcher 3: Wild Hunt             0.000000
4        76561199028326951  The Witcher 3: Wild Hunt             0.000000
...                    ...                       ...                  ...
3999995  76561198055387404     Insurgency: Sandstorm             0.000000
3999996  76561198420472727     Insurgency: Sandstorm             0.000000
3999997  76561198188219363     Insurgency: Sandstorm             0.665501
3999998  76561198408699867     Insurgency: Sandstorm             0.000000
3999999  76561198055317027     Insurgency: Sandstorm             0.493225

[3987414 rows x 3 columns]


In [13]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import SVD
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate
reader = Reader(rating_scale=(0, 1))
data= Dataset.load_from_df(new_dt,reader=reader)
trainSet, testSet = train_test_split(data,test_size=0.2,random_state=60)
algo = SVD()
print(data)
algo.fit(trainSet)
predictions = algo.test(testSet)
accuracy.rmse(predictions)
cross_validate(algo,data,measures=['RMSE','MAE'],cv = 5, verbose=True)


RMSE: 0.2387


In [ ]:
pred = algo.predict(uid=10,iid=1)
print(pred)

user: 10         item: 1          r_ui = None   est = 0.14   {'was_impossible': False}


In [ ]:
import difflib
import random



def get_game_id(book_title, metadata):
    
    """
    Gets the game ID for a game title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata[sv.APP_NAME].values)
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
    book_id = metadata[metadata[sv.APP_NAME] == closest_titles[0]][sv.APP_ID].values[0]
    return book_id

def get_game_info(book_id, metadata):
    
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    
    book_info = metadata[metadata[sv.APP_ID] == book_id][[sv.APP_ID, sv.APP_NAME]]
    return book_info.to_dict(orient='records')

def predict_review(user_id, book_title, model, metadata):
    
    """
    Predicts the review (on a scale of 1-5) that a user would assign to a specific book. 
    """
    
    book_id = get_game_id(book_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=book_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4):
    
    """
    Generates a book recommendation for a user based on a rating threshold. Only
    books with a predicted rating at or above the threshold will be recommended
    """
    
    lista_app = list(df['app_name'].unique())
    random.shuffle(lista_app)
    
    for book_title in lista_app:
        rating = predict_review(user_id, book_title, model, metadata)
        if rating >= thresh:
            book_id = get_game_id(book_title, metadata)
            return get_game_info(book_id, metadata)

In [ ]:

print(generate_recommendation(10,algo,df))

None
